## 'Helper file' used for initial analysis behind jup_gpt4_WHYdata

In [ ]:
from key import test_key, miri_key
from info import principles, whatCR, rewriteR, yesnoCR_turned
import numpy as np
import openai
import time
import pandas as pd
from datasets import load_dataset

openai.api_key = miri_key

yesnoCR = yesnoCR_turned

In [3]:
data_path = "/home/vsb29/rds/hpc-work/project/data/data_file1.json"
data = load_dataset("json", data_files=data_path)
data

Found cached dataset json (/home/vsb29/.cache/huggingface/datasets/json/default-332cd82ced9b110c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 281
    })
})

In [4]:
data['train']['instruction']

["Does the statement overemphasize, or show emotions different from the input story? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement.",
 "Is anything in the statement badly phrased or difficult to follow? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement.",
 "Does the statement give a different impression than the input story of whether the defendant has or wants to self-improve (eg counselling) or prevent the offense from happening again? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement.",
 "Does the statement overemphasize, or show emotions different from the input story? Please answer 'yes' or 'no'. Make sure to think carefully about what is said in the input story VS what is said in the statement.",
 "Does the statement overemphasize, or show emo

In [5]:
for i in range(2):
    inp = data['train']['input'][i]
    print(inp)

### INPUT STORY: My name is Ronald Smith and I hit a guy. I am 32 years old and have never done anything like this. I had been drinking with some friends and then this guy said something very rude to my friend, so I got really angry and lost control for a moment.
### STATEMENT: Your Honor, I stand before you today to take full responsibility for my actions. On the night in question, I had been drinking with some friends when a guy said something very rude to my friend. In my anger, I lost control and hit him. I want to make it clear that I am not offering any excuses for what happened that night. However, I do believe that my lack of criminal record and good character show that this was a momentary lapse in judgement rather than a pattern of behavior. I am pleading guilty as a mitigating factor in the hope that the court will take this into account when sentencing me. I understand the seriousness of my actions and am willing to cooperate with authorities to make things right. Furthermo

In [6]:
gpt4_prompt = "You will be presented with an input story, a statement, a question about the input story or statement, and the correct answer to the question. Please provide an explanation for why this is the correct answer by using quotes from the input story and statement. Here are a few principles that the statement should satisfy, and they might me useful in your explanation: "
explanations = []
data_points = np.array([273, 211, 222,  77, 275, 140, 135, 124, 256,  85,  19, 190, 142, 156, 272, 111, 191,  97, 154,  24])
print(data_points)
for j in range(12, len(data_points)):
    i = data_points[j]
    print(i)
    inp = data['train']['input'][i].split("### ")
    q = data['train']['instruction'][i].split("Please answer 'yes' or 'no'.")[0].strip()
    ans = data['train']['output'][i]
    chat_log = [{"role": "system", "content": gpt4_prompt + principles},
                {"role": "user", "content": inp[1].strip() + "\n" + inp[2] + "\nQUESTION: " + q + "\nANSWER: "+ans + "\nEXPLANATION: "}]
    explanation = answ = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=chat_log
        )
    explanations.append(explanation)

[273 211 222  77 275 140 135 124 256  85  19 190 142 156 272 111 191  97
 154  24]


In [7]:
data_points

array([273, 211, 222,  77, 275, 140, 135, 124, 256,  85,  19, 190, 142,
       156, 272, 111, 191,  97, 154,  24])

In [14]:
for j in range(4):
    i = j+16
    print(data_points[i])
    print("Q: ", data['train']['instruction'][data_points[i]].split("Please answer 'yes' or 'no'.")[0].strip())
    print("A: ", data['train']['output'][data_points[i]])
    print(data['train']['input'][data_points[i]].split("### ")[2])
    print(data['train']['input'][data_points[i]].split("### ")[1].strip())
    print(explanations[j+4]["choices"][0]["message"]["content"])
    print("")

191
Q:  Does the statement give a different impression than the input story of whether the defendant has or wants to self-improve (eg counselling) or prevent the offense from happening again?
A:  yes
STATEMENT: Your Honor, I stand before you today as a defendant, acknowledging my guilt and pleading guilty to the offense of drunk driving. This guilty plea is an essential element that benefits the court and demonstrates my sincere remorse and acceptance of full responsibility for the consequences of my poor judgment. The incident occurred when my 8-year-old child urgently requested to come home after dinner. Unfortunately, in my haste to be there for my child, I made the ill-advised decision to get behind the wheel of my car after consuming a few glasses of wine. I deeply regret my actions and take full responsibility for them. I understand and am fully aware of the severity of my actions and the potential harm they could have caused, not only to myself and my son but also to other innoc

In [ ]:
exp_4_evals = [
    "273, Good. Gets main point, and also more (for another principle)",
    "211, Good enough. Not easy to explain why something is coherent, but does well enough. No quotes.",
    "222, Good. No quotes, but there is 'no misplaced info', so that is not needed either",
    "77, Good. Acknowledges that input has no emotions, but statement has loads.",
    "275, Good. Sees plain mistake in statement (no record, which is false)",
    "140, Good. Explains emotions in story and statement, and how they cohere.",
    "135, Good. Quotes the main reason for the satisfied principle.",
    "124, OK. No amends anywhere, BUT says emotions principle is also good, although this is not the case...",
    "256, Good. Discusses amends although it is in neither story.",
    "85, Perfect. Concise and 100percent correct. ",
    "19, Good. Sees the most important breach, but also goes slightly beyond",
    "190, Good enough. Gives one example, but there is more information that this that is unsupported.",
    "142, Good. (coherence...)",
    "156, Good. Points to several important quotes that exist both in statement and story.",
    "272, Good. Explains in general what is similar, but is missing quotes / more specific examples.",
    "111, No. The not-coherent part is hitting someone unintentionally, but GPT4 does not catch this.",
    "191, Good. Sees concrete examples - but also misses one bad quote.",
    "97, Good. Explains what relevant info is there and what irrelevant info is excluded.",
    "154, Good. (age is broken)",
    "24, No. Disagrees with the given answer!:O But answer is not obvious..."
]

In [ ]:
exp_turbo = []
for i in data_points:
    inp = data['train']['input'][i].split("### ")
    q = data['train']['instruction'][i].split("Please answer 'yes' or 'no'.")[0].strip()
    ans = data['train']['output'][i]
    chat_log = [{"role": "system", "content": gpt4_prompt + principles},
                {"role": "user", "content": inp[1].strip() + "\n" + inp[2] + "\nQUESTION: " + q + "\nANSWER: "+ans + "\nEXPLANATION: "}]
    explanation = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=chat_log
        )
    exp_turbo.append(explanation)

### Notes about using GPT-4

- Explanation may reach further than only the principle in question (eg emotions mentions amends; wanting to better manage anger).
- And discussion around that other principle might not be correct (eg 124)
- When something is not mentioned in either input story or statement, it tends to explain other things
- 24: Statement says "when an altercation arose between one of my friends and a stranger". But acc to story, there was no altercation - just "this guy said something very rude to my friend".

All-in-all GPT-4 was correct with quite good explanations 90% of the time (18/20) (counting the amends which was wrong about emotions as good...)